In [2]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/side_concordance_results.xlsx')

In [3]:
def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'
    
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)


In [4]:
# Define modality
modality = '4D CT Scan'

# Apply extract_side function if not already done to get imaging sides
preop_imaging_df['4D CT Imaging Side'] = preop_imaging_df[modality].apply(extract_side)

# Filter to scans that localized anywhere (Left, Right)
localized_4dct = preop_imaging_df[preop_imaging_df['4D CT Imaging Side'].isin(['Left', 'Right'])].copy()

# Count total localized scans
total_localized_4dct = localized_4dct.shape[0]
total_localized_4dct

171

In [6]:
def check_concordance(row):
    imaging = row['Sestamibi']
    surgical = row['Surgical_Findings']
    
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'
    imaging_glands = [g.strip() for g in imaging.split(',')]
    surgical_glands = [g.strip() for g in surgical.split(',')]
    
    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'
    
    imaging_sides = []
    for ig in imaging_glands:
        if 'Left' in ig:
            imaging_sides.append('Left')
        if 'Right' in ig:
            imaging_sides.append('Right')
    
    surgical_sides = []
    for sg in surgical_glands:
        if 'Left' in sg:
            surgical_sides.append('Left')
        if 'Right' in sg:
            surgical_sides.append('Right')
    
    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'
    
    return 'Incorrect Localization'


In [7]:
localized_4dct['Calculated Concordance'] = localized_4dct.apply(check_concordance, axis=1)

false_positives = (localized_4dct['Calculated Concordance'] == 'Incorrect Localization').sum()
true_positive = (
    (localized_4dct['Calculated Concordance'] == 'Yes (Exact)') |
    (localized_4dct['Calculated Concordance'] == 'Yes (Side)')
).sum()

ppv = true_positive / (true_positive + false_positives)

print(f"Positive Predictive Value (PPV) for localized 4D CT scans: {ppv:.2%}")

Positive Predictive Value (PPV) for localized 4D CT scans: 62.71%
